# <div style="text-align: center">18.335/6.337 Problem Set 4 - Least Squares, Conditioning, and Floating Point</div>
## <div style="text-align: center">Solutions</div>

### <div style="text-align: right">Creaed by Wonseok Shin</div>

$\newcommand{\Cmat}[2]{\mathbb{C}^{#1\times#2}}\newcommand{\Cvec}[1]{\mathbb{C}^{#1}}\newcommand{\Rmat}[2]{\mathbb{R}^{#1\times#2}}\newcommand{\Rvec}[1]{\mathbb{R}^{#1}}\newcommand{\null}{\mathrm{null}}\newcommand{\range}{\mathrm{range}}\newcommand{\rank}{\mathrm{rank}}\newcommand{\nullity}{\mathrm{nullity}}\newcommand{\sign}{\mathrm{sign}}\newcommand{\norm}[1]{\left\|#1\right\|}\newcommand{\abs}[1]{\left|#1\right|}\newcommand{\epsmach}{\epsilon_\mathrm{machine}}\newcommand{\log}{\mathrm{log}}\newcommand{\tanh}{\mathrm{tanh}}\newcommand{\l}{\lambda}\newcommand{\d}{\delta}$

## General instructions
- Please archive all files (this Jupyter notebook plus other files if any) into a single ZIP file and upload it to the course website, so that the grader can run the Jupyter notebook in the extracted folder without adding anything.  (This greatly simplifies the grader's job.)
- Some questions may require proofs or explanations, and there are a few options to submit them electronically.
    1. Ideally, write down your proofs or explanations in the corresponding sections of this Jupyter notebook.  You can use $\LaTeX$ commands for mathematical symbols, like $A = \hat{U} \Sigma V^*$, $\sum_{j = 1}^{r} \sigma_j u_j v_j^*$, and $\left[
\begin{array}{cc}
 1 & 2 \\
 3 & 4 \\
\end{array}
\right]$.
    2. If you are not familiar with $\LaTeX$ commands, you can use any word processors with equation editors.  If you choose to use this option, make sure to convert the word processor file into PDF before submission.
    3. If none of the above works for you, you can also submit a scan of your handwritten notes as a last resort, but your handwriting must be clear and readable.
- The score of bonus problems, if any, will be added to your total problem set score at the end of the semester.  However, your total problem set score cannot exceed the perfect problem set score without bonus problems.  In other words, 
$$
\text{your total p-set score} = \min\left\{\sum_n\text{your $n$th p-set score $with$ bonus problems}, \sum_n\text{perfect $n$th p-set score $without$ bonus problems}\right\}.
$$
So solve bonus problems to compensate for the deducted points in your problem sets.

## Problem 1.  Estimating a "black box" linear system from inputs and outputs [2 pts]

Consider some system inside a "black box" that takes an input $x \in \Cvec{n}$ and generates an output $y \in \Cvec{m}$.  Suppose we know the system is linear, i.e. it is described by some underlying matrix $A$ such that $y = A x$, but unfortunately we cannot open the black box, and the only way to probe $A$ is to supply some input $x$ and measure the corresponding output $y$.  The goal of this problem is to estimate $A$ from a series of such measurements.

(a) [0.5 pts] If the measurements are error-free, how many measurements do we need?  Are there any requirements for these measurements?

***Solution***.

Suppose we have $N$ measurements, and the $j$th measurement consists of the input vector $x_j$ and output vector $y_j$.  Construct matrices $X = \left[ x_1\,\cdots\,x_N\right]$ and $Y = \left[ y_1\,\cdots\,y_N\right]$.  Then we have $Y = A X$, or 

$$
Y^* = X^* A^*
$$

equivalently.  The $i$th column of $A^*$ satisfies 

$$
(Y^*)_{:,i} = X^* (A^*)_{:,i}.
$$

For this equation to determine $(A^*)_{:,i}$ uniquely for all $i$, the equation must not be underdetermined, so the matrix $X^* \in \Cmat{N}{n}$ must have at least $N = n$ rows.  This means we need at least $n$ measurements to determine $A$.

When we have exactly $n$ measurements, $X^*$ has to be invertible to determine $(A^*)_{:,i}$ uniquely.  This is the case when $X^*$ is full-rank.  Therefore, our $n$ input vectors $x_1, \ldots, x_n$ must be linearly independent.

(b) [1 pt] In reality, measurement has error, so each measurement is modeled by $y = A x + v$ for a random error vector $v$.  To "average out" the effect of the random error, we usually perform $N$ measurements with $N$ much greater than the number needed for the error-free case in Part (a).  Then, assuming sufficiently accurate measurement, $A$ minimizing the overall error $\sqrt{\sum_{j=1}^N\norm{v_j}^2}$ would be a good estimate of the system matrix.  (Here, the norm is the 2-norm.)

For given measurement data $y_1, \ldots, y_N$ for $x_1, \ldots, x_N$, describe how to find such $A$.

***Solution***.

Using the setup in the solution to Part (a), we can write $Y = A X + V$ for $V = \left[ v_1\,\cdots\,v_N\right]$, or

$$
V^* = Y^* - X^* A^*
$$

equivalently.  Then, our goal is to minimize

$$
J(A) = \sum_{j=1}^N\norm{v_j}_2^2 = \norm{V}_F^2 = \norm{V^*}_F^2 = \norm{Y^* - X^* A^*}_F^2 = \sum_{i=1}^m \norm{(Y^*)_{:,i} - X^* \, (A^*)_{:,i}}_2^2
$$

over $A$.  

In the last expression, each term in the sum can be individually minimized because $(A^*)_{:,i}$ of $A^*$ is independent of the other columns of $A^*$.  Therefore, we can determine the entries of $A$ minimizing $J(A)$ by solving the least squares problem

$$
\min_{(A^*)_{:,i}} \norm{(Y^*)_{:,i} - X^* \, (A^*)_{:,i}}_2
$$

for $i = 1, \ldots, m$.  For each least squares problem, the matrix is $X^*$, the right-hand-side vector to fit is a column of $Y^*$, and the least squares solution is a column of $A^*$.

(c) [0.5 pt] Using the method you described in Part (b), complete the following code block that estimates $A$ for given measurement data $x_1, \ldots, x_N$ and $y_1, \ldots, y_N$, which are stored as the columns of `X` and `Y`, respectively.

If you need to solve a least squares problem $\min_x \norm{b - A x}$ for some $A$ and $b$ (where $A$ is not necessarily our unknown system matrix $A$), use the backslash operator as `x = A\b`, which solves the least squares problem via QR factorization as documented [here](http://docs.julialang.org/en/stable/stdlib/linalg/#Base.\).

***Solution***.

In [1]:
using JLD
@load "XY.jld"

n, Nx = size(X)
m, Ny = size(Y)
assert(Nx == Ny)
N = Nx

A = Matrix{Float64}(m,n)

Xc = X'
Yc = Y'
for i = 1:m
    ac = Xc \ Yc[:,i]  # ac = A'[:,i]
    A[i,:] = ac'
end

println("Estimate of A:")
A

Estimate of A:


10×3 Array{Float64,2}:
  6.08288    15.0453      15.0142  
  0.0342512  20.976        3.02841 
 21.1033     -0.00762194  20.8107  
 20.9692     11.9143      11.9937  
 27.0083      3.13127     20.9703  
 12.022      11.8812      27.0496  
 14.9444     20.8934      24.0736  
 23.8005     18.242        9.04182 
  0.0282755  26.8861      -0.115509
  3.1801     24.0351      21.0616  

## Problem 2.  Condition number of polynomial root finding [2 pts]

Example 12.5 of Trefethen and Bau evaluates conditioning of polynomial root finding.  Here, the problem is to find a root $\lambda$ of a degree-$m$ polynomial $p(x) = \sum_{k = 0}^m a_k x^k$.  Specifically,

- the problem data are the entire set of coefficients $a_0, \ldots, a_m$, and
- the solution of the problem is $\lambda$.

The condition number of this problem measures sensitivity of $\lambda$ with respect to small perturbation of the problem data.  For simplicity, we assume that all the coeficients except $a_i$ are fixed and perturb only $a_i$.  (At first glance, considering perturbation of a single data component might seem too much simplification, but in fact this is analogous to calculating partial derivatives: like the individual partial derivatives constitute the gradient, considering perturbation of individual data components eventually allows us to picture sensitivity with respect to perturbation of the entire data components.)  The resulting condition number, which describes how much relative change in $\lambda$ is induced by small relative perturbation in $a_i$, is Eq. (12.7):

$$
\kappa = \frac{\abs{\d\l \big/ \l}}{\abs{\d a_i \big/ a_i}} = \frac{\abs{a_i \l^{i-1}}}{\abs{p'(\l)}}.
$$

(a) [0.5 pts] Derive the above condition number.  (Hint: proof of Thm. 12.2.)

***Solution***.

The equation that holds for the perturbed polynomial is $p(\l + \d \l) + \d a_i (\l + \d \l)^i = 0$, or equivalently

$$
p(\l + \d \l) = -\d a_i (\l + \d \l)^i.
$$

The left-hand side expands into the Taylor series as

$$
p(\l + \d \l) = p(\l) + p'(\l) \d \l + O((\d \l)^2) = p'(\l) \d \l + O((\d \l)^2)
$$

because $p(\l) = 0$, and the right-hand side expands as

$$
-\d a_i (\l + \d \l)^i = -\l^i \d a_i + \d a_i O(\d \l).
$$

Equating the two, we get

$$
p'(\l) \d \l + O((\d \l)^2)  = -\l^i \d a_i + \d a_i O(\d \l).
$$

Note that the right-hand side collects all the $\d a_i$-dependent terms.  Therefore, this equation describes how much change in $\d \l$ (left-hand side) is induced by active perturbation $\d a_i$ (right-hand side).

Because perturbation $\d a_i$ and the induced change $\d \l$ are infinitesimally small, we can keep only the most dominant term on each side to get

$$
p'(\l) \d \l = -\l^i \d a_i,
$$

from which we obtain

$$
\kappa = \frac{\abs{\d\l \big/ \l}}{\abs{\d a_i \big/ a_i}} = \frac{\abs{\d\l \big/ \d a_i}}{\abs{\l \big/ a_i}} = \frac{\abs{\l^i\big/p'(\l)}}{\abs{\l\big/a_i}} = \frac{\abs{a_i \l^{i-1}}}{\abs{p'(\l)}}.
$$

(b) [1 pt] The above condition number diverges if $\lambda$ is a multiple root because $p^\prime(\lambda) = 0$.  In other words, the solution is infinitely sensitive to small perturbation of a coefficient.  However, even in that case it turns out that $k$th-order sensitivity

$$
\kappa^{(k)} = \frac{\abs{\d\l \big/ \l}^k}{\abs{\d a_i \big/ a_i}} 
$$

does not diverge, where $k$ is multiplicity of the root $\lambda$.

Derive the expression for $\kappa^{(k)}$ that can be evaluated when $p(x)$ and $\lambda$ are given.

***Solution***.

Again, the equation that holds for the perturbed polynomial is

$$
p(\l + \d \l) = -\d a_i (\l + \d \l)^i,
$$

but this time the left-hand side expands into the Taylor series as

$$
\begin{align}
p(\l + \d \l) &= \left[p(\l) + p'(\l) \d\l + \frac{1}{2} p''(\l) (\d\l)^2 + \cdots + \frac{1}{(k-1)!} p^{(k-1)} (\d\l)^{k-1}\right] + \frac{1}{k!} p^{(k)}(\l) (\d\l)^k + O((\d \l)^{k+1})\\
& = \frac{1}{k!} p^{(k)}(\l) (\d\l)^k + O((\d \l)^{k+1}),
\end{align}
$$

whose most dominant term is $\frac{1}{k!} p^{(k)}(\l) (\d\l)^k$.  Equating this with the most dominant term on the right-hand side of the first equation, we get

$$
\frac{1}{k!} p^{(k)}(\l) (\d\l)^k = -\d a_i \l^i,
$$

from which we obtain

$$
\kappa^{(k)} = \frac{\abs{\d\l \big/ \l}^k}{\abs{\d a_i \big/ a_i}} = \frac{\abs{k!\d a_i \l^{i-k}/p^{(k)}(\l)}}{\abs{\d a_i \big/ a_i}} = \frac{\abs{k! a_i \l^{i-k}}}{\abs{p^{(k)}(\l)}}.
$$

(c) [0.5 pts] Validate your result in Part (b) for perturbation of $p(x) = x^2 - 2x + 1$ to $x^2 - 2x + 0.9999$.

***Solution***.

In this example, $a_0 = 1$ is perturbed to $a_0 - 0.0001$, so $i = 0$ and $\abs{\d a_i/a_i} = 0.0001$.  

The polynomials before and after perturbation are $(x-1)^2$ and $(x-1.01)(x-0.99)$, respectively.  The polynomial  before perturbation has a root $\l = 1$ with multiplicity $k = 2$, and $p^{(k)}(\l) = 2$.  The polynomial after perturbation has roots $\l \pm 0.01$, so $\abs{\d\l/\l} = 0.01$ for both roots.

With these parameters, we have

$$
\frac{\abs{\d\l \big/ \l}^k}{\abs{\d a_i \big/ a_i}} = \frac{0.01^2}{0.0001} = 1
$$

and

$$
\frac{\abs{k! a_i \l^{i-k}}}{\abs{p^{(k)}(\l)}} = \frac{2! \cdot 1 \cdot 1^{0-2}}{2} = 1.
$$

Because these two are the same, the example validates the expression of $\kappa^{(k)}$ derived in Part (b).

## Problem 3.  Floating-point peculiarity [3 pts]

In this problem, we will see some peculiar behaviors of floating-point numbers and try to understand them.

A few things to keep in mind:

- As will be discussed in class, the bit representation of IEEE double-precision floating point numbers is $S\,E_1 \cdots E_{11} M_1 \cdots M_{52}$, where
    - a single bit $S$ stores the sign,
    - 11 bits $E_1 \cdots E_{11}$ store the exponent information, and 
    - 52 bits $M_1 \cdots M_{52}$ store the mantissa (or significand) information.
- `eps()` in Julia returns the gap between 1 and the next larger number in the double-precision floating-point system.  
    - This is *not* the "machine epsilon" $\epsmach = \frac{1}{2}\beta^{-(t-1)}$ defined in Eq. (13.3) of Trefethen and Bau, but is twice as big, i.e., `eps()` $= \beta^{-(t-1)}$.
    - Trefethen and Bau's definition of machine epsilon is not universal: many other authors take the value of `eps()` as machine epsilon, as shown [here](https://en.wikipedia.org/wiki/Machine_epsilon#Values_for_standard_hardware_floating_point_arithmetics).
- `bin()` in Julia is useful for obtaining the binary representation of integers; for example, `bin(23)` returns `"10111"`.

(a) [0.5 pts] Find the bit representation of the value returned by `eps()`.  (Indicate the values of the sign bit $S$, exponent bits $E_1 \cdots E_{11}$, and the mantissa bits $M_1 \cdots M_{52}$.)

***Solution***.

`eps()` is $(-1)^0 \times 1.0 \cdots 0_{(2)} \times 2^{-52} = (-1)^s m \beta^e$.  Therefore, 
- $S = 0$, 
- $M_1 \cdots M_{52} = 0 \cdots 0$, and 
- $E_1 \cdots E_{11}$ is the binary representation of $-52 + 1023$, which is $01111001011$ as shown below.

In [2]:
bin(-52+1023)

"1111001011"

(b) [0.5 pts] Evaluate `1+eps()-1` and `2+eps()-2`.  Explain why you get different results.

***Solution***.

`1+eps()-1` and `2+eps()-2` return `eps()` and `0`, respectively.  To explain this result, note that
- the mantissa of $1+\mathrm{eps()} = 1.\underbrace{0 \cdots 0 1}_{\text{52 bits}} \times 2^0$ requires 52 bits to store, whereas
- the mantissa of $2+\mathrm{eps()} = 1.\underbrace{0 \cdots 0 1}_{\text{53 bits}} \times 2^1$ requires 53 bits to store.

Therefore, the mantissa of `2+eps()` needs to be truncated to be stored in the 52 mantissa bits of the double-precision floating-point format.  As a result, `2+eps()` is approximated as

$$
2+\mathrm{eps()} = 1.\underbrace{0 \cdots 0 1}_{\text{53 bits}} \times 2^1 \approx 1.\underbrace{0 \cdots 0}_{\text{52 bits}} \times 2^1 = 2,
$$

and therefore `(2+eps())-2` produces `2-2 = 0`.

(c) [0.5 pts] Evaluate `1+eps()/2-1` and `1-1+eps()/2`.  Explain why you get different results.

***Solution***.

`1+eps()/2-1` and `1-1+eps()/2` return `0` and `eps()/2`, respectively.  The latter is obvious.  To explain the former, note that again, the mantissa of $1+\mathrm{eps()}/2 = 1.\underbrace{0 \cdots 0 1}_{\text{53 bits}} \times 2^0$ requires 53 bits to store, so it needs to be truncated to be stored in the 52 mantissa bits of the double-precision floating-point format.  As a result, `1+eps()/2` is approximated by `1`, and `(1+eps()/2)-1` produces `1-1=0`.

(d) [0.5 pts] Evaluate `(2^-537.)^2` and `(2^-538.)^2`.  Explain why you get 0 from the latter.

***Solution***.

In exact arithmetic, `(2^-537.)^2` and `(2^-538.)^2` are $2^{-1074}$ and $2^{-1076}$, respectively.  The former is the smallest denormalized number of the double-precision floating-point format, whereas the latter is smaller than that.  Therefore, the latter suffers from underflow and is approximated by 0.

(e) [0.5 pts] (Taken from Exercise 13.2b)  The double-precision floating-point system includes many integers, but not all of them.  What is the smallest positive integer $n$ that does not belong to this system?  

***Solution***.

Because the double-precision floating-point system has 52 mantissa bits (except for the implicit leading bit $M_0 = 1$), the positive integers that do not belong to the system are the ones that require more than 52 mantissa bits except for the leading bit.  The smallest of them is $1\underbrace{0\cdots 01}_{\text{53 bits}} = 2^{53} + 1$.

(f) [0.5 pts] (Taken from Exercise 13.2c)  Verify your answer in Part (e) by Julia.  Write Julia code such that

- it shows $n-3, n-2, n-1$ belong to the double-precision floating-point system, but $n$ does not, and 
- it shows whether or not $n+1$, $n+2$, $n+3$ belong to the double-precision floating-point system.

After running the code, explain the results for $n+1$, $n+2$, $n+3$.

***Solution***.

For $n = 2^{53} + 1 = 1\underbrace{0\cdots 01}_{\text{53 bits}} = 1.\underbrace{0\cdots 01}_{\text{53 bits}}\times 2^{53}$,
- $n+1 = 1\underbrace{0\cdots 1}_{\text{52 bits}}0 = 1.\underbrace{0\cdots 01}_{\text{52 bits}}\times 2^{53}$ requires 52 mantissa bits,
- $n+2 = 1\underbrace{0\cdots 11}_{\text{53 bits}} = 1.\underbrace{0\cdots 11}_{\text{53 bits}}\times 2^{53}$ requires 53 mantissa bits, and
- $n+3 = 1\underbrace{0\cdots 1}_{\text{51 bits}}00 = 1.\underbrace{0\cdots 01}_{\text{51 bits}}\times 2^{53}$ requires 51 mantissa bits.

Therefore, within 52 mantissa bits of the IEEE double-precision floating-point format, $n+1$ and $n+3$ fit, but $n$ and $n+2$ don't.  The following code validates this theory:

In [3]:
function test_n(n)
    println("n-3 == float(n-3)?  $(n-3==float(n-3))")
    println("n-2 == float(n-2)?  $(n-2==float(n-2))")
    println("n-1 == float(n-1)?  $(n-1==float(n-1))")    
    println("  n == float(n)?    $(n==float(n))")
    println("n+1 == float(n+1)?  $(n+1==float(n+1))")
    println("n+2 == float(n+2)?  $(n+2==float(n+2))")
    println("n+3 == float(n+3)?  $(n+3==float(n+3))")
end

test_n(2^53+1)

n-3 == float(n-3)?  true
n-2 == float(n-2)?  true
n-1 == float(n-1)?  true
  n == float(n)?    false
n+1 == float(n+1)?  true
n+2 == float(n+2)?  false
n+3 == float(n+3)?  true


## Problem 4.  Numerical evaluation of $e^x - 1$ and $\log(1+x)$ [3 pts]

In this problem, we will observe numerical issues in evaluating $e^x-1$ and $\log(1+x)$, and devise ways to avoid them.

Let's first consider the condition number.  Out problem is to evaluate either $f(x) = e^x - 1$ or $f(x) = \log(1+x)$ for problem data $x$.  However, the conventional *relative* condition number (note the use of $\abs{\cdot}$ instead of $\norm{\cdot}$ for scalar $x$ and $f(x)$ below)

$$
\kappa(x) = \sup_{\d x}\frac{\abs{f(x+\d x) - f(x)}\big/\abs{f(x)}}{\abs{\d x}\big/\abs{x}}
$$

is not well-defined at $x = 0$ because both $\abs{f(x)}$ and $\abs{x}$ are zero there.

To work around this difficulty, we will consider the *absolute* condition number (Eq. (12.2) of Trefethen and Bau):

$$
\hat{\kappa}(x) = \sup_{\d x}\frac{\abs{f(x+\d x) - f(x)}}{\abs{\d x}}.
$$

The absolute condition number is well-defined for our $f$ even at $x = 0$.

(a) [0.5 pts] Derive the analytic expression of $\hat{\kappa}(x)$ for $f(x) = e^x - 1$.  What is $\hat{\kappa}(0)$ according to your analytic expression?

Also, evaluate $\abs{f(x+\d x) - f(x)}/\abs{\d x}$ numerically in Julia for small perturbation $\delta x <$ `eps()` at $x = 0$.  You may expect a value close to the previous analytic result, but it will be significantly different.  Explain what numerical issue causes this error.

***Solution***.

We have
$$
\hat{\kappa}(x) = \sup_{\delta x}\frac{\abs{e^{x+\d x} - e^x}}{\abs{\delta x}} = e^x \sup_{\delta x}\frac{\abs{e^{\d x} - 1}}{\abs{\delta x}} = e^x \lim_{\delta x \rightarrow 0}\abs{\frac{e^{\d x} - 1}{\delta x}} = e^x,
$$

so $\hat{\kappa}(0) = 1$.

Howover, for $\d x < \mathrm{eps}()$, say $\d x = \mathrm{eps}()/2$, we get $\abs{f(0+\d x) - f(0)}/\abs{\d x} = 0$ numerically; see the code below.  

The numerical issue causing this result is catastrophic cancellation between $e^x$ and $1$, but we can elaborate it as follows.  Because the slope of $e^x$ is 1 at $x = 0$, $e^{\d x}$ must be approximately $1 + \d x$.  For $\d x < \mathrm{eps}()$, $1 + \d x$ requires more than 52 mantissa bits to be stored accurately, so $e^{\d x}$ is approximated by 1 (see Prob. 3c).  As a result, we have 

$$
\begin{align}
f(0+\d x) &= e^{\d x} - 1\\ 
&\approx (1+\d x) - 1\\
&\approx 1 - 1 = 0,
\end{align}
$$

so we get the wrong result $\abs{f(0+\d x) - f(0)}/\abs{\d x} \approx 0$.

In [4]:
f(x) = exp(x) - 1
x = 0.
δx = eps()/2
abs((f(x+δx) - f(x))/δx)

0.0

(b) [1 pt] Devise a way to evaluate $f(x) = e^x - 1$ without the above numerical issue.  (Hint: $\tanh(x)$.)  

Also, verify $f(x)$ evaluated this way produces $\abs{f(x+\delta x) - f(x)}/\abs{\delta x}$ consistent with $\hat{\kappa}(0)$.

***Solution***.

Because $\tanh(x) = (e^x - e^{-x}) / (e^x + e^{-x}) = (e^{2x}-1) / (e^{2x}+1)$, we have $\tanh(x/2) = (e^x-1) / (e^x+1)$.  Therefore,

$$
e^x-1 = \tanh\left(\frac{x}{2}\right) (e^x + 1).
$$

The right-hand side expression avoids catastrophic cancellation between $e^x$ and $1$ for $x \approx 0$.  Therefore, it produces accurate results even for small x, and $\abs{f(0+\d x) - f(0)}/\abs{\d x}$ evaluated with the right-hand side expression produces the value close to $\hat{\kappa}(0) = 1$; see the code below.

In [5]:
f(x) = tanh(x/2) * (exp(x)+1)
x = 0.
δx = eps()/2
abs((f(x+δx) - f(x))/δx)

1.0

(c) [0.5 pts] Repeat Part (a) for $f(x) = \log(1+x)$, where $\log$ is the natural logarithm.

***Solution***.

We have
$$
\hat{\kappa}(x) = \sup_{\delta x}\frac{\abs{\log(1+x+\d x) - \log(1+x)}}{\abs{\delta x}} = \frac{1}{\abs{1+x}} \sup_{\delta x} \abs{\frac{\log\left(1+\frac{\d x}{1+x}\right)}{\frac{\delta x}{1+x}}} = \frac{1}{\abs{1+x}} \lim_{\delta x \rightarrow 0} \abs{\frac{\log\left(1+\frac{\d x}{1+x}\right)}{\frac{\delta x}{1+x}}} = \frac{1}{\abs{1+x}},
$$

so $\hat{\kappa}(0) = 1$.

However, for $\d x < \mathrm{eps}()$, say $\d x = \mathrm{eps}()/2$, we get $\abs{f(0+\d x) - f(0)}/\abs{\d x} = 0$ numerically; see the code below.  

The numerical issue causing this result can be described as follows.  In this case, $f(0+\d x) = \log(1+\d x)$.  Then, for $\d x < \mathrm{eps}()$, $1+\d x$ is approximated by $1$ (see Prob. 3c).  Therefore, $f(0+\d x) = \log(1 + \d x) \approx \log(1) = 0$, so we get the wrong result $\abs{f(0+\d x) - f(0)}/\abs{\d x} \approx 0$.

In [6]:
f(x) = log(1+x)
x = 0.
δx = eps()/2
abs((f(x+δx) - f(x))/δx)

0.0

(d) [1 pt] Repeat Part (b) for $f(x) = \log(1+x)$.  (Hint: $\tanh^{-1}(x)$.)

***Solution***.

Note $\tanh^{-1}(z) = \frac{1}{2} \log\left(\frac{1+z}{1-z}\right) = \frac{1}{2} \log\left(1 + \frac{2z}{1-z}\right)$ for $-1 < z < 1$.  Using the change of variables $x = \frac{2z}{1-z} \Longleftrightarrow z = \frac{x}{x+2}$, we have

$$
\log(1+x) = 2\tanh^{-1}\left(\frac{x}{x+2}\right).
$$

Note that $x > -1$ for the domain $-1 < z < 1$ of $\tanh^{-1}(z)$, so the above equation holds for all $x > -1$, which is the entire domain of $\log(1+x)$.  Unlike the left-hand side's argument $1+x$ that is truncated to constant $1$ for small $x$, the right-hand side's argument $\frac{x}{x+2} = x \cdot \frac{1}{x+2}$ has the small $x$ factored out, so its approximation $\frac{x}{x+2} \approx \frac{x}{2}$ for small x still captures variation of small $x$.  Therefore, the right-hand side expression produces accurate results even for small x, and $\abs{f(0+\d x) - f(0)}/\abs{\d x}$ evaluated with the right-hand side expression produces the value close to $\hat{\kappa}(0) = 1$; see the code below.

In [7]:
f(x) = 2atanh(x/(x+2))
x = 0.
δx = eps()/2
abs((f(x+δx) - f(x))/δx)

1.0

## Bonus problem.  Oblique projection [3 pts]

We used an orthogonal projector to find the solution to the least squares problem $\min_x \norm{b - A x}_2$.  What if we use an oblique projector for some reason?  Of course, the resulting solution $x$ will no longer minimize $\norm{b - A x}_2$, but still this is an interesting problem to think about.  (Who knows?  You may find a cool application of oblique projection in the future.)

The orthogonal projector we used to solve the least squares problem was $P = A (A^* A)^{-1} A^*$.  It projects a vector onto $\range(A)$, and the projection directions are orthogonal to $\range(A)$.  Because the directions orthogonal to $\range(A)$ must be orthogonal to the columns of $A$, for these projection directions $d$ we have $A^* d = 0$, or $d \in \null(A^*)$ equivalently.  In other words, the orthogonal projector projects onto $\range(A)$ along $\null(A^*)$.

For oblique projection, we still want to project onto $\range(A)$, but now the projection directions are not orthogonal to $\range(A)$; instead, they are orthogonal to the range of another matrix $B$.  This means that the projection directions of this oblique projector are in $\null(B^*)$.

(a) [1 pt] Find the expression (in terms of $A$ and $B$) of the oblique projector $P'$ that projects onto $\range(A)$ along $\null(B^*)$, where $A, B \in \Cmat{m}{n}$ are full-rank and skinny.  Assume $\range(A) \cap \null(B^*) = \{0\}$.  (Hint: like we did for the orthogonal projector in class, make a good guess of $P'$ first and then verify it.)

***Solution***.

From the conditions, we have $\range(P') = \range(A)$ and $\null(P') = \null(B^*)$.  Therefore, we can guess that $P'$ is in the form of

$$
P' = A T B^*
$$

for some $T \in \Cmat{n}{n}$.  If this guess is right, because $P'$ is a projector, $T$ must satisfy $P'^2 = 
P'$, or

$$
A (T B^* A T) B^* = A T B^*.
$$

The obvious way to satisfy the above equality is to satisfy $T B^* A T = T$.  If $T$ is invertible, then this condition means $T = (B^* A)^{-1}$.  So, our final guess is

$$
P' = A (B^* A)^{-1} B^*.
$$

The only remaining steps are to show if $P'$ is indeed a projector we are looking for:
- First note that $B^* A$ is invertible, because $\range(A) \cap \null(B^*) = {0}$; otherwise, $B^* A$ would be rank-deficient and have nonzero $x$ as a nullspace element, which would mean $B^*$ has a nonzero vector $A x$ as a nullspace element (because $A$ is skinny and full-rank), which would contradict the condition that $B$ is full-rank.  Because $B^* A$ is invertible, $P'$ is well-defined.  
- It is trivial to verify $P'^2 = P'$, so $P'$ is indeed a projector.  
- Because $(B^* A)^{-1} B^*$ is full-rank and fat, $\range((B^* A)^{-1} B^*) = \Cvec{n}$, so $\range(A (B^* A)^{-1} B^*) = \range(A)$.  Similarly, because $A (B^* A)^{-1}$ is full-rank and skinny, $\null(A (B^* A)^{-1}) = {0}$, so $\null(A (B^* A)^{-1} B^*) = \null(B^*)$.

Therefore, indeed $P'$ is a projector projecting onto $\range(A)$ along $\null(B^*)$.

(b) [1 pt] For $A$ and $B$ in Part (a) and for $b \in \Cvec{m}$, find
- the unique $x \in \Cvec{n}$ such that $A x$ is the oblique projection point of $b$ onto $\range(A)$ along $\null(B^*)$,
- a set of linear equations (like the normal equation) whose solution is $x$, and 
- the relationship between $A$ and $B$ for which the equation becomes ill-conditioned.

***Solution***.

We want to find $x$ such that

$$
A x = P' b = A (B^* A)^{-1} B^* b.
$$

The right-hand side of the equation is in $\range(A)$, so the above equation has a solution.  Also, because $A$ is full-rank, the solution is unique.  One obvious solution is 

$$
x = (B^* A)^{-1} B^* b, 
$$

so this is the unique solution we are looking for.

The system of linear equation that has this $x$ as the solution is

$$
B^* A x =  B^* b.
$$

When $\range(A)$ contains a direction $v = A x$ that is nearly orthogonal to $\range(B)$, then $B^* v$ becomes nearly zero.  This means that the minimum singular value of $B^*A$ becomes very close to zero and the condition number of the matrix becomes very large.  Then, the equation becomes ill-conditioned.  Therefore, if $\range(A)$ has some direction that is nearly orthogonal to $\range(B)$, the above equation becomes ill-conditioned.

(c) [1 pt] Describe an algorithm (like Alg. 11.2) that finds $x$ of Part (b).  Use two QR factorizations and one matrix-matrix multiplication as the only steps whose operation counts depend cubically on $m$ and $n$ combined (e.g., $\sim m n^2$ or $n^3$).  In addition, find the asymptotic expression for the total operation count.

***Solution***.

First, compute the reduced QR factorization of $B = \hat{Q}_B \hat{R}_B$.  Then, because $\hat{R}_B$ is invertible for the full-rank $B$, the equation $B^* A x = B^* b$ reduces to

$$
\hat{Q}_B^* A x = \hat{Q}_B^* b.
$$

Now, define $C = \hat{Q}_B^* A$ and $c = \hat{Q}_B^* b$.  We can solve the equation $C x = c$ by performing the full QR factorization of $C = Q_C R_C$.  (Note that $C$ is square, so its full and reduced QR factorizations are the same.)

In summary, the algorithm is as follows:
1. Compute the reduced QR factorization $B = \hat{Q}_B \hat{R}_B$.
2. Compute $C = \hat{Q}_B^* A$ and $c = \hat{Q}_B^* b$.
3. Compute the full QR factorization $C = Q_C R_C$.
4. Compute $Q_C^* c$.
5. Solve $R_C x = Q_C^* c$ by back substitution.

Only Steps 1−3 are the steps whose operation counts depend cubically on $m$ and $n$: 
- Step 1 takes $2 m n^2 - \frac{2}{3} n^3$ flops by the Householder QR factorization.  
- Step 2 takes $2n (2m - n)$ flops per column of $A$ (see Prob. 1.3a of PS3), so  it takes $2n^2 (2m-n)$ flops in total.  
- Step 3 takes $2 n^3 - \frac{2}{3} n^3 = \frac{4}{3} n^3$ flops by the Householder QR factorization.

Therefore, the entire algorithm takes $6 m n^2 - \frac{4}{3} n^3$ flops asymptotically.